In [1]:
# Three machine learning models will be trained and tested to classify whether the neutron star is pulsar or not:
# Naive Bayes, Linear Discriminant Analysis and Logistic Regression.


In [2]:
# Import relevant libraries.
import matplotlib 
matplotlib.use('Agg')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from dmba import classificationSummary
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    accuracy_score,
    f1_score,
    confusion_matrix, 
    precision_recall_fscore_support,
    roc_curve, 
    accuracy_score, 
    roc_auc_score)

/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_6601/1504372894.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


no display found. Using non-interactive Agg backend


In [3]:
df = pd.read_csv('pulsar_stars.csv') # import the dataset.

In [4]:
df.info() # check general information about the dataset.
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17898 entries, 0 to 17897
Data columns (total 9 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0    Mean of the integrated profile                17898 non-null  float64
 1    Standard deviation of the integrated profile  17898 non-null  float64
 2    Excess kurtosis of the integrated profile     17898 non-null  float64
 3    Skewness of the integrated profile            17898 non-null  float64
 4    Mean of the DM-SNR curve                      17898 non-null  float64
 5    Standard deviation of the DM-SNR curve        17898 non-null  float64
 6    Excess kurtosis of the DM-SNR curve           17898 non-null  float64
 7    Skewness of the DM-SNR curve                  17898 non-null  float64
 8   target_class                                   17898 non-null  int64  
dtypes: float64(8), int64(1)
memory usage: 1.2 MB


,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,0
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,0


In [5]:
# rename columns for them to have shorter names
df.columns = ['mean_profile', 'std_profile', 'kurtosis_profile', 'skewness_profile', 'mean_dmsnr',
               'std_dmsnr', 'kurtosis_dmsnr', 'skewness_dmsnr', 'target']

In [6]:
df.describe()

,mean_profile,std_profile,kurtosis_profile,skewness_profile,mean_dmsnr,std_dmsnr,kurtosis_dmsnr,skewness_dmsnr,target
count,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000,17898.000000
mean,111.079968,46.549532,0.477857,1.770279,12.614400,26.326515,8.303556,104.857709,0.091574
std,25.652935,6.843189,1.064040,6.167913,29.472897,19.470572,4.506092,106.514540,0.288432
min,5.812500,24.772042,-1.876011,-1.791886,0.213211,7.370432,-3.139270,-1.976976,0.000000
25%,100.929688,42.376018,0.027098,-0.188572,1.923077,14.437332,5.781506,34.960504,0.000000
50%,115.078125,46.947479,0.223240,0.198710,2.801839,18.461316,8.433515,83.064556,0.000000
75%,127.085938,51.023202,0.473325,0.927783,5.464256,28.428104,10.702959,139.309330,0.000000
max,192.617188,98.778911,8.069522,68.101622,223.392141,110.642211,34.539844,1191.000837,1.000000


In [7]:
plt.figure(figsize=(5,5))
plt.pie(df["target"].value_counts().values,labels=["Not pulsars","Pulsars"], autopct="%1.0f%%")
plt.title("Proportion of target variable in dataset")
plt.show()
plt.savefig('Pie_chart.png')

print('Pulsar?\n', '0 - no\n', '1 - yes\n', df['target'].value_counts()) # Check the target_class to understand what is the ratio between pulsars and other neutron stars.
print("Pulsar ratio:", (df['target'].value_counts()[1]/np.sum(df['target'].value_counts())*100).round(2), "%")

Pulsar?
 0 - no
 1 - yes
 target
0    16259
1     1639
Name: count, dtype: int64
Pulsar ratio: 9.16 %


/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_6601/1272216001.py:4: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [8]:
plt.figure(figsize = (14,12)) # Check correlations of variables via correlation heatmap.
mask = np.triu(np.ones_like(df.corr()))
sns.heatmap(df.corr(), cmap = 'RdYlBu', mask = mask, annot = True)
plt.show()
plt.savefig('corr_heatmap.png')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/seaborn/matrix.py:260: FutureWarning: Format strings passed to MaskedConstant are ignored, but in future may error or produce different behavior
  annotation = ("{:" + self.fmt + "}").format(val)
/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_6601/2459305105.py:4: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [9]:
# Plot histograms for each variable and their corresponding QQ-plots
# to check if the distributuins are normal.

name_columns = list(df.columns[:-1])  # remove the last element of the list, as it will serve as a classifier.
label = ['target_class']

colors = ["red", "orange", "navy", "green", "blue", "purple", "black", "magenta"]  # different colors for different plots.

fig, axes = plt.subplots(8, 2, figsize=(20, 14))  # set the number of rows, columns and figure size.

for i, var in enumerate(name_columns):  # plot the distribution of all of the classifier variables.
    ax_distribution = axes[i, 0]  # Select the current axis for distribution plot.
    ax_qq = axes[i, 1]  # Select the current axis for QQ plot.
    c = colors[i]  # Assign the color.

    # Plot distribution.
    ax_distribution.hist(df[var], bins=50, density=True, label=var, color=c, linestyle='-', histtype='step')
    ax_distribution.legend()  # Add legend for each histogram.
    ax_distribution.set_xlabel(var)
    ax_distribution.set_ylabel('Density')

    # Plot QQ-plot
    stats.probplot(df[var], dist="norm", plot=ax_qq)
    ax_qq.set_title('QQ Plot for ' + var)
    ax_qq.set_xlabel('Theoretical Quantiles')
    ax_qq.set_ylabel('Sample Quantiles')

plt.tight_layout()
plt.show()
plt.savefig('distribution_QQ.png')

/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_6601/1336107018.py:29: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [10]:
# As we can see from the plots, none of the metrics have perfect normal distribution.
# However, a lot of metrics have somewhat skewed Gaussian distribution, or some outliers are present.

# The Gaussian Naive Bayes model will be used for the data in this case. However, such classifier 
# may not perform to a reasonable extent, as the Gaussian distribution is not followed in all of the metrics.

In [11]:
# GAUSSIAN NAIVE BAYES

predictors = list(df.columns[0:-1]) # variables on which the model will be trained. Includes all columns except the last one, as it will be value which will need to be predicted.
X = df[predictors]
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0) # Build Gaussian Naive Bayes based on 8 predictors, leaving 15% of data for testing purposes.
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test) # fit the model.
print("Numbers of elements in training subset:", X_train.shape[0]) # print out the relevant output after model fitting.
print("Numbers of elements in testing subset:", X_test.shape[0])
print("Number of mislabeled points out of a total %d points : %d"
      % (X_test.shape[0], (y_test != y_pred).sum()))
percent_error = (y_test != y_pred).sum() / X_test.shape[0]
print("The percentage of mislabeled points is:", percent_error.round(4)*100,'%')

Numbers of elements in training subset: 15213
Numbers of elements in testing subset: 2685
Number of mislabeled points out of a total 2685 points : 116
The percentage of mislabeled points is: 4.32 %


In [12]:
# Additional metrics upon which model can be evaluated
print("Mean Squared Error (MSE):", mean_squared_error(y_test, y_pred).round(3))
print("Mean Absolute Error (MAE):", mean_absolute_error(y_test, y_pred).round(3))
print("Accuracy:", accuracy_score(y_pred, y_test))
print("F1 score:", f1_score(y_pred, y_test, average="weighted"))

classificationSummary(y_train, gnb.predict(X_train),
                     class_names=gnb.classes_)

Mean Squared Error (MSE): 0.043
Mean Absolute Error (MAE): 0.043
Accuracy: 0.9567970204841714
F1 score: 0.9545028455821191
Confusion Matrix (Accuracy 0.9437)

       Prediction
Actual     0     1
     0 13145   639
     1   218  1211


In [13]:
fpr, tpr, thresholds = roc_curve(y, gnb.predict_proba(X)[:, 0], # create an Reciever Operating Curve (ROC) for the Gaussian Naive Bayes.
                                pos_label = 0)
roc_df = pd.DataFrame({'recall': tpr, 'specificity': 1 - fpr}) # Create a dataframe with values for recall and specificity.

ax = roc_df.plot(x = 'specificity', y = 'recall', figsize = (4,4), legend = False) # plot the ROC.
ax.set_ylim(0,1)
ax.set_xlim(1,0)
ax.plot((1,0), (0,1))
ax.set_xlabel('specificity')
ax.set_ylabel('recall')
plt.tight_layout()
plt.show()
plt.savefig('gaussian_.png')

/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_6601/1912420246.py:12: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [14]:
print("Area under the ROC curve:", roc_auc_score([1 if yi == 0 else 0 for yi in y], gnb.predict_proba(X)[:, 0]).round(3)) # Calculate the area under the ROC.

Area under the ROC curve: 0.956


In [15]:
# LINEAR DISCRIMINANT ANALYSIS.

predictors = list(df.columns[0:-1])
outcome = 'target'

df_train, df_test = train_test_split(df, test_size=0.15, random_state=123)

X = df_train[predictors]
y = df_train[outcome]

## get the predicted label from the linear discriminant.
## Use the test sample for using the model:
lda = LinearDiscriminantAnalysis()
lda.fit(X, y)

# Print the scaling factors by which variable data will be multiplied for better performance.
# These scaling factors are learned from the data during the training phase of the LDA model.
print(pd.DataFrame(lda.scalings_, index = X.columns)) 

                         0
mean_profile      0.023253
std_profile      -0.012235
kurtosis_profile  3.121369
skewness_profile -0.217042
mean_dmsnr       -0.007532
std_dmsnr         0.022770
kurtosis_dmsnr   -0.063782
skewness_dmsnr    0.002205


In [16]:
pred = pd.DataFrame(lda.predict_proba(df_train[predictors]), # create dataframe with LDA calculated predictions for the training dataset.
                   columns=lda.classes_)
print(pred.head())

          0             1
0  0.999977  2.287774e-05
1  0.999981  1.912261e-05
2  0.999998  1.774387e-06
3  1.000000  3.308020e-09
4  1.000000  4.441415e-07


In [17]:
# Plot the LDA output.
plt.hist(pred[1], bins = 20, density = True, label = 'sig', linewidth = 1.5, color = 'red', linestyle = '-', histtype = 'step')
plt.hist(pred[0], bins = 20, density = True, label = 'bkg', linewidth = 1.5, color = 'blue', linestyle = '-', histtype = 'step')
plt.xlabel('LDA output')
plt.ylabel('Number of Occurance')
plt.legend()
plt.show()
plt.savefig('lda_output.png')

/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_6601/3413030614.py:7: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [18]:
classificationSummary(y, lda.predict(X),
                     class_names=lda.classes_)

Confusion Matrix (Accuracy 0.9757)

       Prediction
Actual     0     1
     0 13769    56
     1   313  1075


In [19]:
fpr, tpr, thresholds = roc_curve(y, lda.predict_proba(X)[:, 0], # create an Reciever Operating Curve (ROC) for the Linear Discriminant Analysis.
                                pos_label = 0)
roc_df = pd.DataFrame({'recall': tpr, 'specificity': 1 - fpr}) # Create a dataframe with values for recall and specificity.

ax = roc_df.plot(x = 'specificity', y = 'recall', figsize = (4,4), legend = False) # plot the ROC.
ax.set_ylim(0,1)
ax.set_xlim(1,0)
ax.plot((1,0), (0,1))
ax.set_xlabel('specificity')
ax.set_ylabel('recall')
plt.tight_layout()
plt.show()
plt.savefig('lda_roc.png')

/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_6601/3613028858.py:12: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [20]:
print("Area under the ROC curve:", roc_auc_score([1 if yi == 0 else 0 for yi in y], lda.predict_proba(X)[:, 0]).round(3))

Area under the ROC curve: 0.975


In [21]:
# LOGISTIC REGRESSION

X = pd.get_dummies(df_train[predictors], prefix='', prefix_sep='', # Each variable is converted in as many 0/1 variables as there are different values.
                  drop_first = True, dtype = 'int') 
y = df_train[outcome]

logit_reg = LogisticRegression(penalty='l2', solver = 'liblinear') # l2 penalty uses the sum of the squares of the parameters, enables smaller but non-zero coefficients.
logit_reg.fit(X, y) # Fit the logistic regression.

print('intercept', logit_reg.intercept_[0])
print('classes', logit_reg.classes_)
pd.DataFrame({'coeff': logit_reg.coef_[0]}, 
            index = X.columns)

intercept -3.532782518420596
classes [0 1]


,coeff
mean_profile,0.007020
std_profile,-0.046322
kurtosis_profile,5.494674
skewness_profile,-0.531294
mean_dmsnr,-0.028927
std_dmsnr,0.032680
kurtosis_dmsnr,-0.252148
skewness_dmsnr,0.004642


In [22]:
pred = pd.DataFrame(logit_reg.predict_proba(X), # create dataframe with predictions from logistic regression.
                   columns = logit_reg.classes_)

print(pred)

plt.hist(pred[1], bins = 50, density = True, label = 'Pulsar', linewidth = 1.5, color = 'red', linestyle = '-', histtype = 'step')
plt.hist(pred[0], bins = 50, density = True, label = 'Not Pulsar', linewidth = 1.5, color = 'blue', linestyle = '-', histtype = 'step')
plt.xlabel('Logistic regression probability')
plt.ylabel('Number of Occurance')
plt.legend()
plt.show()

              0         1
0      0.956542  0.043458
1      0.963568  0.036432
2      0.993049  0.006951
3      0.998546  0.001454
4      0.996238  0.003762
...         ...       ...
15208  0.992286  0.007714
15209  0.989212  0.010788
15210  0.997317  0.002683
15211  0.989108  0.010892
15212  0.994090  0.005910

[15213 rows x 2 columns]


/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_6601/3022201299.py:11: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [27]:
import statsmodels.api as sm

y_numbers = [1 if yi == 0 else 0 for yi in y]
logit_reg_sm = sm.GLM(y_numbers, X.assign(const = 1), 
                     family=sm.families.Binomial())

logit_result = logit_reg_sm.fit()
print(logit_result.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                15213
Model:                            GLM   Df Residuals:                    15204
Model Family:                Binomial   Df Model:                            8
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1064.2
Date:                Tue, 23 Apr 2024   Deviance:                       2128.3
Time:                        12:40:33   Pearson chi2:                 3.38e+04
No. Iterations:                     8   Pseudo R-squ. (CS):             0.3755
Covariance Type:            nonrobust                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
mean_profile        -0.0325      0.007  

In [24]:
# Create Confusion Matrix

classificationSummary(y, logit_reg.predict(X),
                     class_names=logit_reg.classes_)

Confusion Matrix (Accuracy 0.9801)

       Prediction
Actual     0     1
     0 13756    69
     1   233  1155


In [25]:
fpr, tpr, thresholds = roc_curve(y, logit_reg.predict_proba(X)[:, 0],
                                pos_label = 0)
roc_df = pd.DataFrame({'recall': tpr, 'specificity': 1 - fpr})

ax = roc_df.plot(x = 'specificity', y = 'recall', figsize = (4,4), legend = False)
ax.set_ylim(0,1)
ax.set_xlim(1,0)
ax.plot((1,0), (0,1))
ax.set_xlabel('specificity')
ax.set_ylabel('recall')
plt.tight_layout()
plt.show()  
plt.savefig('figure.png')
plt.savefig('log_roc.png')

/var/folders/hx/c878nf113fv7h9m8j458sdbh0000gn/T/ipykernel_6601/3932584376.py:12: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [26]:
print("Area under the ROC curve:", roc_auc_score([1 if yi == 0 else 0 for yi in y], logit_reg.predict_proba(X)[:, 0]).round(3))

Area under the ROC curve: 0.976
